# 03-Nature Tutorial- Ground State Solvers

#### Uncomment the cell below to pip install the necessary modules if not already installed

#### Note: Works with Qiskit Version 1.4.1 and Quantum Rings Qiskit Toolkit Version 0.1.10

In [1]:
# %pip install qiskit-nature
# %pip install qiskit-algorithms
# %pip install quantumrings-toolkit-qiskit==0.1.10
# %pip install qiskit==1.4.1

#### Restart the kernel after installing any of the missing packages

### Introduction

In this notebook tutorial, we are going to discuss the ground state calculation interface of Qiskit Nature. The goal is to compute the ground state of a molecular Hamiltonian. This Hamiltonian can be electronic or vibrational. To learn more about the preparation of the Hamiltonian, check out the notebooks on Electronic Structure and Vibrational Structure.

> One thing to note, in the electronic case, we are actually computing purely the electronic part. When using the Qiskit Nature stack as presented in this tutorial, the *nuclear repulsion energy* will be added automatically, to obtain the **total** ground state energy.

The first step is to define the molecular system. In the following we ask for the electronic part of a hydrogen molecule. This will follow a similar procedure as seen in the previously mentioned notebooks.

In [2]:
# This code is from:
# https://qiskit-community.github.io/qiskit-nature/tutorials/03_ground_state_solvers.html

In [3]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()


We will be sticking to the Jordan-Wigner mapping. To learn more about the various mappers available in Qiskit Nature, check out the [Qubit Mappers tutorial](https://qiskit-community.github.io/qiskit-nature/tutorials/06_qubit_mappers.html).

In [4]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()

### The Solver

Now we need to define a solver. The solver is the algorithm through which the ground state is computed.

Let's first start with a purely classical example: the `NumPyMinimumEigensolver`. This algorithm exactly diagonalizes the Hamiltonian. Although it scales badly, it can be used on small systems to check the results of the quantum algorithms.

In [5]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()

To find the ground state we could also use the Variational Quantum Eigensolver (VQE) algorithm. The VQE algorithm works by exchanging information between a classical and a quantum computer as depicted in the following figure.

![VQE](aux_files/vqe.png)

Now let's initialize a VQE solver.

In [6]:
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from quantumrings.toolkit.qiskit import QrEstimatorV1 as Estimator

from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(Estimator(), ansatz, COBYLA())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

To define the VQE solver one needs three essential elements:

1. An `Estimator` primitive: Here we import the `Estimator` from Quantum Rings Qiskit Toolkit
2. A variational form: Here we used the Unitary Coupled Cluster (UCC) ansatz. For reference, see [Quantum Algorithms for Electronic Structure](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.022322). Since it is a chemistry standard, a factory is already available allowing a fast initialization of a VQE with UCC. The default is to use all single and double excitations. However, the excitation type (S, D, SD) as well as other parameters can be selected. We also preprend the `UCCSD` variational form with a `HartreeFock` initial state, which initializes the occupation of our qubits according to the problem which we are trying to solve.
3. An optimizer: This is the classical piece of code in charge of optimizing the parameters in our variational form. Feel free to check out [more optimizers](https://qiskit-community.github.io/qiskit-algorithms/apidocs/qiskit_algorithms.optimizers.html).

One could also use any available ansatz/initial state or even define one's own. For instance, we define a `TwoLocal` ansatz below with different parameters. Feel free to look at the [TwoLocal](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.TwoLocal) documentation.

In [7]:
from qiskit_algorithms import VQE
from qiskit.circuit.library import TwoLocal

tl_circuit = TwoLocal(
    rotation_blocks=["h", "rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=2,
    parameter_prefix="y",
)

another_solver = VQE(Estimator(), tl_circuit, COBYLA())

### The calculation and results

We are now ready to put everything together to compute the ground-state of our problem. Doing so requires us to wrap our `mapper` and quantum algorithm into a single `GroundStateEigensolver` as shown below.

In [12]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe_solver)

Let's recap the workflow:

1. Generate the second-quantized operators stored in our problem (here referred to as `es_problem`)
2. Mapping (and potentially reducing) the operators in the qubit space
3. Running the quantum algorithm on the Hamiltonian qubit operator
4. Once converged, evaluating the additional observables at the determined ground state

In [13]:
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275006879
  - computed part:      -1.857275006879
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.13730601243
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701266]
    - computed part:      [0.0  0.0  1.388948701266]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001266]  Total: 0.000000001266
                 (debye): [0.0  0.0  -0.000000003218]  Total: 0.000000003218
 


We can compare the VQE results to the NumPy exact solver and see that the energies match.

In [14]:
calc = GroundStateEigensolver(mapper, numpy_solver)
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701555]
    - computed part:      [0.0  0.0  1.388948701555]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001555]  Total: 0.000000001555
                 (debye): [0.0  0.0  -0.000000003953]  Total: 0.000000003953
 


### Using a filter function

Sometimes the true ground state of the Hamiltonian is not of interest because it lies in a different symmetry sector of the Hilbert space. In this case, the `NumPyEigensolver` can take a filter function to return only the eigenstates with, for example, the correct number of particles. The function may act to give us only the information we want to know. This is of particular importance in the case of vibrational structure calculations where the true ground state of the Hamiltonian is the vacuum state. A default filter function to check the number of particles is implemented in the different problems and its use is shown in the cell below.

In [16]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.drivers import GaussianForcesDriver
from qiskit_nature.second_q.mappers import DirectMapper
from qiskit_nature.second_q.problems import HarmonicBasis

driver = GaussianForcesDriver(logfile="aux_files/CO2_freq_B3LYP_631g.log")
basis = HarmonicBasis([2, 2, 2, 2])
vib_problem = driver.run(basis=basis)
vib_problem.hamiltonian.truncation_order = 2

mapper = DirectMapper()

solver_without_filter = NumPyMinimumEigensolver()
solver_with_filter = NumPyMinimumEigensolver(
    filter_criterion=vib_problem.get_default_filter_criterion()
)

gsc_wo = GroundStateEigensolver(mapper, solver_without_filter)
result_wo = gsc_wo.solve(vib_problem)

gsc_w = GroundStateEigensolver(mapper, solver_with_filter)
result_w = gsc_w.solve(vib_problem)

print(result_wo)
print("\n\n")
print(result_w)

The optional dependency 'sparse' is not installed. Falling back to using 'numpy' arrays instead. Consider installing the 'sparse' package to reduce memory requirements.


=== GROUND STATE ===
 
* Vibrational ground state energy (cm^-1): (-3e-12-0j)
The number of occupied modals for each mode is: 
- Mode 0: 0.0
- Mode 1: 0.0
- Mode 2: 0.0
- Mode 3: 0.0



=== GROUND STATE ===
 
* Vibrational ground state energy (cm^-1): 2432.10695403655
The number of occupied modals for each mode is: 
- Mode 0: 1.0
- Mode 1: 1.0
- Mode 2: 1.0
- Mode 3: 1.0


### Conclusion

Congratulations! You have explored new ways of solving for the ground state of a molecule. We looked at the instantiation of a solver, and hinted at other possible solvers to use. We also discussed the use of multiple ansatz, and introduced the idea of a filter function to give us relevant information. Understanding how to calculate the ground state of a molecule is important for when we want to go on to solve for the excited states, as the methods there will build on the methods of the ground state.